In [ ]:
project_path = "/home/jupyter"
import os
import sys
sys.path.append(project_path)
sys.path.append(f'{project_path}/ft_events/src/utils')

from google.cloud import bigquery
import importlib

import numpy as np
import pandas as pd
from plotly import graph_objs as go
import seaborn as sns

import matplotlib.dates as mdates
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import plotly.express as px

from fintrans_toolbox.src import table_utils as t


client = bigquery.Client()

28th July - 8th August 2022

Hosted at Birmingham, West Midlands region, and London.

birmingham 

alexander stadium - b42 2lr

arena birmigham/utilita - b1 2aa

edgbaston - b5 7qu

smithfield - b5

sutton park - b74 


wmr 

coventry stadium CV6

Victoria park CV31

national exhibition centre - b40

london - e40 7,000ppl

In [ ]:
cw_areas = ['B', 'CV']

In [ ]:
sql_spend = f"""SELECT time_period_value, merchant_location_level,
merchant_location, cardholder_issuing_country, mcg, mcc, spend, transactions, cardholders
FROM ons-fintrans-data-prod.fintrans_visa.spend_merchant_location
WHERE time_period = 'Month' AND
merchant_location_level != 'All' AND
cardholder_issuing_level != 'All' 
GROUP BY time_period_value, merchant_location_level, cardholder_issuing_country, merchant_location, mcg, mcc, spend, transactions, cardholders
ORDER BY time_period_value, merchant_location_level, cardholder_issuing_country, merchant_location, mcg, mcc, spend, transactions, cardholders"""

sml_df = client.query(sql_spend).to_dataframe()
sml_df = t.create_date_time(sml_df)

In [ ]:
cw_df = sml_df[(sml_df['merchant_location'].isin(cw_areas)) & (sml_df['merchant_location_level']=='POSTAL_AREA') &
                (sml_df['mcg']=='All')].copy()

In [ ]:
def add_nations_periods(fig):

    # define covid lockdown start and end dates
    game_periods = [
        ("2022-07-28", "2022-08-08")
    ]
    # change colour of the covid lockdowns
    fillcolor = "grey"

    # add a dummy trace so that we can add the covid periods to the legend
    # add markers to square to make it show as a square in the legend.
    fig.add_trace(
        go.Scatter(
            x=[None],
            y=[None],
            mode="markers",
            marker=dict(color=fillcolor, symbol="square"),
            name="Commonwealth Games",
        )
    )

    # add covid periods to plot
    for start_date, end_date in game_periods:
        fig.add_shape(
            type="rect",
            xref="x",
            yref="paper",
            x0=start_date,
            y0=0,
            x1=end_date,
            y1=1,
            fillcolor=fillcolor,
            opacity=0.5,
            layer="below",
            line_width=0,
        )

    return fig

In [ ]:
cw_df = cw_df.groupby(['date_time', "merchant_location"]).agg({"spend" : "sum", "transactions" : "sum", "cardholders" : "sum"}).reset_index()


In [ ]:
metrics = ['spend', 'transactions', 'cardholders']

for metric in metrics:
    fig = px.line(
    cw_df,
    x="date_time",
    y=metric,
    color = 'merchant_location',
    title=f"{metric}",
    )
    fig = add_nations_periods(fig)
    fig.show()

## International spending

In [ ]:
cw_df = sml_df[(sml_df['merchant_location'].isin(cw_areas)) & (sml_df['merchant_location_level']=='POSTAL_AREA') &
                (sml_df['mcg']=='All')].copy()

In [ ]:
for area in cw_areas:
    fig = px.line(
    cw_df[(cw_df['merchant_location'] == area) & (cw_df['cardholder_issuing_country'] != 'UNITED KINGDOM') & (cw_df['cardholder_issuing_country'] != 'All')],
    x="date_time",
    y="spend",
    color = 'cardholder_issuing_country',
    title=f"Spend in {area}",
    )
    fig = add_nations_periods(fig)
    fig.show()